In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/car-number-plate/images/N148.xml
/kaggle/input/car-number-plate/images/N177.xml
/kaggle/input/car-number-plate/images/N40.jpeg
/kaggle/input/car-number-plate/images/N176.jpeg
/kaggle/input/car-number-plate/images/N173.xml
/kaggle/input/car-number-plate/images/N129.jpeg
/kaggle/input/car-number-plate/images/N114.jpeg
/kaggle/input/car-number-plate/images/N231.jpeg
/kaggle/input/car-number-plate/images/N69.jpeg
/kaggle/input/car-number-plate/images/N213.xml
/kaggle/input/car-number-plate/images/N119.xml
/kaggle/input/car-number-plate/images/N212.jpeg
/kaggle/input/car-number-plate/images/N52.jpeg
/kaggle/input/car-number-plate/images/N103.xml
/kaggle/input/car-number-plate/images/N154.jpeg
/kaggle/input/car-number-plate/images/N166.xml
/kaggle/input/car-number-plate/images/N127.xml
/kaggle/input/car-number-plate/images/N162.xml
/kaggle/input/car-number-plate/images/N131.jpeg
/kaggle/input/car-number-plate/images/N198.xml
/kaggle/input/car-number-plate/images/N118.jpeg
/kagg

In [7]:
import os
import cv2
import tensorflow as tf
import pytesseract as pt
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet

from glob import glob
from skimage import io
from shutil import copy
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [27]:
filename = df['filepath'][0]
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('/kaggle/input/images',filename_image)
    return filepath_image
getFilename(filename)

'/kaggle/input/images/N148.jpeg'

In [28]:
image_path = list(df['filepath'].apply(getFilename))
image_path

['/kaggle/input/images/N148.jpeg',
 '/kaggle/input/images/N177.jpeg',
 '/kaggle/input/images/N173.jpeg',
 '/kaggle/input/images/N213.jpeg',
 '/kaggle/input/images/N119.jpeg',
 '/kaggle/input/images/N103.jpeg',
 '/kaggle/input/images/N166.jpeg',
 '/kaggle/input/images/N127.jpeg',
 '/kaggle/input/images/N162.jpeg',
 '/kaggle/input/images/N198.jpeg',
 '/kaggle/input/images/N211.jpeg',
 '/kaggle/input/images/N30.jpeg',
 '/kaggle/input/images/N42.jpeg',
 '/kaggle/input/images/N205.jpeg',
 '/kaggle/input/images/N199.jpeg',
 '/kaggle/input/images/N112.jpeg',
 '/kaggle/input/images/N187.jpeg',
 '/kaggle/input/images/N188.jpeg',
 '/kaggle/input/images/N178.jpeg',
 '/kaggle/input/images/N91.jpeg',
 '/kaggle/input/images/N79.jpeg',
 '/kaggle/input/images/N192.jpeg',
 '/kaggle/input/images/N21.jpeg',
 '/kaggle/input/images/N124.jpeg',
 '/kaggle/input/images/N131.jpeg',
 '/kaggle/input/images/N233.jpeg',
 '/kaggle/input/images/N25.jpeg',
 '/kaggle/input/images/N128.jpeg',
 '/kaggle/input/images/N22

In [29]:
path = glob('/kaggle/input/car-number-plate/images/*.xml')
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filename in path:

    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)

In [30]:
df = pd.DataFrame(labels_dict)
df.head()

,filepath,xmin,xmax,ymin,ymax
0,/kaggle/input/car-number-plate/images/N148.xml,244,369,240,293
1,/kaggle/input/car-number-plate/images/N177.xml,331,538,263,317
2,/kaggle/input/car-number-plate/images/N173.xml,80,335,150,243
3,/kaggle/input/car-number-plate/images/N213.xml,131,209,129,153
4,/kaggle/input/car-number-plate/images/N119.xml,180,559,216,314


In [34]:
img_arr = cv2.imread(image_path[0])

[ WARN:0@1777.349] global loadsave.cpp:248 findDecoder imread_('/kaggle/input/images/N148.jpeg'): can't open/read file: check file path/integrity


In [35]:
labels = df.iloc[:,1:].values
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    print(img_arr)
    img_arr=np.array(img_arr)
    h,w,d = img_arr.shape
    # Prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # Normalization
    # Normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # Normalized output
    # Append
    data.append(norm_load_image_arr)
    output.append(label_norm)

None


[ WARN:0@2126.394] global loadsave.cpp:248 findDecoder imread_('/kaggle/input/images/N148.jpeg'): can't open/read file: check file path/integrity


ValueError: not enough values to unpack (expected 3, got 0)

In [ ]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape